In [1]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r "/content/drive/MyDrive/mmmmm/" .

In [4]:
!git clone https://github.com/tensorflow/tpu/

Cloning into 'tpu'...
remote: Enumerating objects: 10814, done.
remote: Counting objects: 100% (1016/1016), done.
remote: Compressing objects: 100% (498/498), done.
remote: Total 10814 (delta 551), reused 909 (delta 495), pack-reused 9798
Receiving objects: 100% (10814/10814), 45.73 MiB | 28.52 MiB/s, done.
Resolving deltas: 100% (7589/7589), done.


In [5]:
from flask import Flask
from flask_ngrok import run_with_ngrok

import cv2
from datetime import datetime

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

# Ini bagian inisialisasi-nya, ndak usah dirubah-rubah ndak apa-apa

from flask import url_for, render_template, request
from IPython import display
from PIL import Image
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
import sys
sys.path.insert(0, 'tpu/models/official')
sys.path.insert(0, 'tpu/models/official/mask_rcnn')
import coco_metric
from mask_rcnn.object_detection import visualization_utils

from keras.preprocessing.image import img_to_array

ID_MAPPING = {
    1: 'orang',
    2: 'sepeda pancal',
    3: 'mobil',
    4: 'sepeda motor',
    5: 'pesawat terbang',
    6: 'bus',
    7: 'kereta',
    8: 'truk',
    9: 'perahu',
    10: 'bangjo',
    11: 'pemadam',
    13: 'rambu berhenti',
    14: 'meteran parkir',
    15: 'bangku',
    16: 'burung',
    17: 'kucing',
    18: 'anjing',
    19: 'kuda',
    20: 'domba',
    21: 'sapi',
    22: 'gajah',
    23: 'beruang',
    24: 'zebra',
    25: 'jerapah',
    27: 'ransel',
    28: 'payung',
    31: 'tas tangan',
    32: 'dasi',
    33: 'koper',
    34: 'frisbee',
    35: 'ski',
    36: 'papan luncur salju',
    37: 'bola',
    38: 'layang-layang',
    39: 'tongkat baseball',
    40: 'sarung tangan baseball',
    41: 'skateboard',
    42: 'papan selancar',
    43: 'raket tenis',
    44: 'botol',
    46: 'gelas anggur',
    47: 'cangkir',
    48: 'garpu',
    49: 'pisau',
    50: 'sendok',
    51: 'mangkuk',
    52: 'pisang',
    53: 'apel',
    54: 'sandwich',
    55: 'jeruk',
    56: 'brokoli',
    57: 'wortel',
    58: 'hot dog',
    59: 'pizza',
    60: 'donat',
    61: 'kue',
    62: 'kursi',
    63: 'sofa',
    64: 'tanaman pot',
    65: 'tempat tidur',
    67: 'meja makan',
    70: 'toilet',
    72: 'tv',
    73: 'laptop',
    74: 'mouse',
    75: 'remot',
    76: 'keyboard',
    77: 'hp',
    78: 'microwave',
    79: 'oven',
    80: 'pemanggang roti',
    81: 'bak cuci',
    82: 'kulkas',
    84: 'buku',
    85: 'jam',
    86: 'vas bunga',
    87: 'gunting',
    88: 'boneka beruang',
    89: 'pengering rambut',
    90: 'sikat gigi',
}
category_index = {k: {'id': k, 'name': ID_MAPPING[k]} for k in ID_MAPPING}
use_tpu = True
if use_tpu:
  import os
  import pprint

  assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)

  session = tf.Session(TPU_ADDRESS, graph=tf.Graph())
  print('TPU devices:')
  pprint.pprint(session.list_devices())
else:
  session = tf.Session(graph=tf.Graph())
saved_model_dir = 'gs://cloud-tpu-checkpoints/mask-rcnn/1555659850'
_ = tf.saved_model.loader.load(session, ['serve'], saved_model_dir)


#-----------------------------------------------------------
#CNN
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.metrics import categorical_accuracy


#------------------------------------------------------------

def apaya(alamat):
  with open(alamat, 'rb') as f:
    gambar_string = np.array([f.read()])
    f.close()

  image = Image.open(alamat)
  width, height = image.size
  np_image = np.array(image.getdata()).reshape(height, width, 3).astype(np.uint8)

  num_detections, detection_boxes, detection_classes, detection_scores, detection_masks, image_info = session.run(
      ['NumDetections:0', 'DetectionBoxes:0', 'DetectionClasses:0', 'DetectionScores:0', 'DetectionMasks:0', 'ImageInfo:0'],
      feed_dict={'Placeholder:0': gambar_string})

  # Capek juga nulis komentar
  # wkwkwkwk
  # Ini ndak perlu diutik-utik ga apa-apa, nanti kalau udah tingkat 4 di SC dibahas lagi deh ya
  num_detections = np.squeeze(num_detections.astype(np.int32), axis=(0,))
  detection_boxes = np.squeeze(detection_boxes * image_info[0, 2], axis=(0,))[0:num_detections]
  detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
  detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections]
  instance_masks = np.squeeze(detection_masks, axis=(0,))[0:num_detections]
  ymin, xmin, ymax, xmax = np.split(detection_boxes, 4, axis=-1)
  processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)
  segmentations = coco_metric.generate_segmentation_from_masks(instance_masks, processed_boxes, height, width)


  detection_boxes = detection_boxes[detection_scores>0.25]
  instance_masks = instance_masks[detection_scores>0.25]
  segmentations = segmentations[detection_scores>0.25]
  detection_classes = detection_classes[detection_scores>0.25]
  detection_scores = detection_scores[detection_scores>0.25]


  detection_boxes = detection_boxes[detection_classes==1]
  detection_scores = detection_scores[detection_classes==1]
  instance_masks = instance_masks[detection_classes==1]
  segmentations = segmentations[detection_classes==1]
  detection_classes = detection_classes[detection_classes==1]

  if detection_boxes.shape[0] == 0:
    return detection_boxes, 0

  d = detection_boxes.astype(int)
    
  tinggi = abs(np.average(d[:,2]-d[:,0]))
  pengali = 170/tinggi
  Cx = (d[:,0]+d[:,2])//2
  Cy = (d[:,1]+d[:,3])//2
  cen_x = np.average(Cx)
  cen_y = np.average(Cy)

  jarak_rata2 = np.average(np.sqrt((Cx-cen_x)**2+(Cy-cen_y)**2))
  jarak_rata2 = jarak_rata2*pengali//100

  max_boxes_to_draw = 300
  min_score_thresh = 0.1
  image_with_detections = visualization_utils.visualize_boxes_and_labels_on_image_array(
      np_image,
      detection_boxes,
      detection_classes,
      detection_scores,
      category_index,
      instance_masks=segmentations,
      use_normalized_coordinates=False,
      max_boxes_to_draw=max_boxes_to_draw,
      min_score_thresh=min_score_thresh)
  output_image_path = './static/wkwk/test_results.jpg'
  Image.fromarray(image_with_detections.astype(np.uint8)).save(output_image_path)

  return detection_boxes, jarak_rata2
#&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
# ini buat mendeteksi objek, sehingga harus di run jika ada gambar baru

def apasih(alamat):
  nama = {0: 'ga', 1: 'pake'}

  with open(alamat, 'rb') as f:
    np_image_string = np.array([f.read()])
    f.close()

  image = Image.open(alamat)
  width, height = image.size
  np_image = np.array(image.getdata()).reshape(height, width, 3).astype(np.uint8)

  num_detections, detection_boxes, detection_classes, detection_scores, detection_masks, image_info = session.run(
      ['NumDetections:0', 'DetectionBoxes:0', 'DetectionClasses:0', 'DetectionScores:0', 'DetectionMasks:0', 'ImageInfo:0'],
      feed_dict={'Placeholder:0': np_image_string})

  num_detections = np.squeeze(num_detections.astype(np.int32), axis=(0,))
  detection_boxes = np.squeeze(detection_boxes * image_info[0, 2], axis=(0,))[0:num_detections]
  detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
  detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections]
  instance_masks = np.squeeze(detection_masks, axis=(0,))[0:num_detections]
  ymin, xmin, ymax, xmax = np.split(detection_boxes, 4, axis=-1)
  processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)
  segmentations = coco_metric.generate_segmentation_from_masks(instance_masks, processed_boxes, height, width)


  detection_boxes = detection_boxes[detection_classes==1]
  detection_scores = detection_scores[detection_classes==1]
  instance_masks = instance_masks[detection_classes==1]
  segmentations = segmentations[detection_classes==1]
  detection_classes = detection_classes[detection_classes==1]


  detection_boxes = detection_boxes[detection_scores>0.5]
  instance_masks = instance_masks[detection_scores>0.5]
  segmentations = segmentations[detection_scores>0.5]
  detection_classes = detection_classes[detection_scores>0.5]
  detection_scores = detection_scores[detection_scores>0.5]


  #buat ngefilter orang aja

  detection_boxes = detection_boxes[detection_classes==1]
  detection_scores = detection_scores[detection_classes==1]
  instance_masks = instance_masks[detection_classes==1]
  segmentations = segmentations[detection_classes==1]
  detection_classes = detection_classes[detection_classes==1]

  #kepala
  detection_boxes[:,2]=detection_boxes[:,0]+(detection_boxes[:,3]-detection_boxes[:,1])//2

  #ini buat ngerubah kotak yang floating jadi integer
  d = detection_boxes.astype(int)

  #buat memotong orang
  font = cv2.FONT_HERSHEY_SIMPLEX 
  fontScale = 1
  color = (255, 255, 255) 
  thickness = 1




  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~`
  size_ = 128

  model = Sequential()

  model.add(Conv2D(16, (3, 3), input_shape = (size_, size_, 3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Conv2D(32, (3, 3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Conv2D(64, (3, 3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Conv2D(128, (3, 3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Conv2D(256, (3, 3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))

  model.add(Flatten())

  model.add(Dense(units = 10000, activation = 'relu'))
  model.add(Dense(units = 100, activation = 'relu'))
  model.add(Dense(units = 1, activation = 'sigmoid'))

  model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

  model.load_weights('/content/drive/MyDrive/Banana Lassi/kkk_7.h5')
  model._make_predict_function()

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

  yang_ga_pake_masker = 0
  for kotak in d :
    '''
    pjg = kotak[2]-kotak[0]
    lbr = kotak[3]-kotak[1]
    lolz = min(pjg, lbr)
    if lolz < 128:
      continue
    '''
    cv2.rectangle(np_image, (kotak[1], kotak[0]), (kotak[3], kotak[2]), (255,255,255), 1)

    img = np_image[kotak[0]:kotak[2],kotak[1]:kotak[3]]
    img = cv2.resize(img, (128, 128)) 
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    hasil = model.predict_classes(img)
    if hasil[0][0] == 0:
      yang_ga_pake_masker = yang_ga_pake_masker + 1
    #hasil = [[1]]

    np_image = cv2.putText(np_image, nama[hasil[0][0]], (kotak[1], kotak[0]), font, fontScale, color, thickness, cv2.LINE_AA) 
  
  np_image = cv2.cvtColor(np_image, cv2.COLOR_BGR2RGB)
  cv2.imwrite('./static/wkwk/test_results.jpg', np_image)

  return str(yang_ga_pake_masker)
#&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
def ini_apa_sih(path):
  with open(path, 'rb') as f:
    # Formatnya Tensorflow
    np_image_string = np.array([f.read()])
    
  image = Image.open(path)
  width, height = image.size

  np_image = np.array(image.getdata()).reshape(height, width, 3).astype(np.uint8)

  num_detections, detection_boxes, detection_classes, detection_scores, detection_masks, image_info = session.run(
    ['NumDetections:0', 'DetectionBoxes:0', 'DetectionClasses:0', 'DetectionScores:0', 'DetectionMasks:0', 'ImageInfo:0'],
    feed_dict={'Placeholder:0': np_image_string})

  num_detections = np.squeeze(num_detections.astype(np.int32), axis=(0,))
  detection_boxes = np.squeeze(detection_boxes * image_info[0, 2], axis=(0,))[0:num_detections]
  detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
  detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections]
  instance_masks = np.squeeze(detection_masks, axis=(0,))[0:num_detections]
  ymin, xmin, ymax, xmax = np.split(detection_boxes, 4, axis=-1)
  processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)
  segmentations = coco_metric.generate_segmentation_from_masks(instance_masks, processed_boxes, height, width)

  max_boxes_to_draw = 300   
  min_score_thresh = 0.1    

  image_with_detections = visualization_utils.visualize_boxes_and_labels_on_image_array(
      np_image,
      detection_boxes,
      detection_classes,
      detection_scores,
      category_index,
      instance_masks=segmentations,
      use_normalized_coordinates=False,
      max_boxes_to_draw=max_boxes_to_draw,
      min_score_thresh=min_score_thresh)
  output_image_path = './static/wkwk/test_results.jpg'
  Image.fromarray(image_with_detections.astype(np.uint8)).save(output_image_path)
  display.display(display.Image(output_image_path, width=1024))

  return 'lol'

#===========================

#===========================

#kotak = detection_boxes.astype(int)

@app.route("/")
def hello():
  return render_template('halo.html')

@app.route('/ini_apa_upload', methods = ['GET', 'POST'])
def ini_apa_upload():
   if request.method == 'POST':
      f = request.files['file']
      path = './static/images' + url_for('hello') + '_' + f.filename
      path_gray = './static/imagesgray' + url_for('hello') + '_' + f.filename
      path_ini = 'static/images' + url_for('hello') + '_' + f.filename
      f.save(path)
      image_ = cv2.imread(path)
      #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      cv2.imwrite(path,image_)

      #return path
      #nama_ini = ini_apa_sih(path)
      d, r = apaya(path)
      
      return render_template(
        'hal3.html',
        ini = str(r),
        posisi = './static/wkwk/test_results.jpg'
      )  

#===========================
@app.route('/apa_ini_upload', methods = ['GET', 'POST'])
def apa_ini_upload():
   if request.method == 'POST':
      f = request.files['file']
      path = './static/images' + url_for('hello') + '_' + f.filename
      path_gray = './static/imagesgray' + url_for('hello') + '_' + f.filename
      path_ini = 'static/images' + url_for('hello') + '_' + f.filename
      f.save(path)
      image_ = cv2.imread(path)
      #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      cv2.imwrite(path,image_)

      #return path
      nama_ini = apasih(path)
      
      return render_template(
        'hal3.html',
        ini = nama_ini,
        posisi = './static/wkwk/test_results.jpg'
      )  

if __name__ == '__main__':
    app.run()

TensorFlow 1.x selected.


Using TensorFlow backend.


TPU address is grpc://10.12.212.10:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11310442418582433582),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10335961133420519672),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12879187894361362553),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 627836803294936810),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17003517881321051166),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15609858862983621287),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 2477731659905279756),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 985812802319492540),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 8203137830482

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c9e6-35-225-43-197.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2022-05-28 04:16:53,356] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-5-0855ce179519>", line 379, in hello
    return render_template('halo.html')
  File "/usr/l

In [6]:
!cp './templates/hal2.html' './templates/hal3.html'

cp: cannot stat './templates/hal2.html': No such file or directory


In [7]:
from flask import Flask
from flask_ngrok import run_with_ngrok

import cv2
from datetime import datetime

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

# Ini bagian inisialisasi-nya, ndak usah dirubah-rubah ndak apa-apa

from flask import url_for, render_template, request

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.metrics import categorical_accuracy

size_ = 128

model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape = (size_, size_, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(256, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(units = 10000, activation = 'relu'))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.load_weights('/content/drive/MyDrive/Banana Lassi/kkk_7.h5')
model._make_predict_function()

img = cv2.imread('/content/static/images/_masker2.jpg')
img = cv2.resize(img, (128, 128)) 
img = img_to_array(img)
img = np.expand_dims(img, axis = 0)
hasil = model.predict_classes(img)
print(hasil)

@app.route("/")
def hello():
  return str(hasil)

if __name__ == '__main__':
    app.run()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


AttributeError: ignored

In [ ]:
from google.colab.patches import cv2_imshow

lol = cv2.rectangle(np_image, (0,0), (100,100), (255, 255, 255), 1)
cv2_imshow(lol)

In [ ]:
lol = cv2.imread('wkwk.jpg')
for d in kotak:
  lol = cv2.rectangle(lol, (d[1],d[0]), (d[3],d[2]), (255, 255, 255), 1)
cv2_imshow(lol)

cv2.imwrite(date_time+'.jpg', lol)

In [ ]:
wkwk = cv2.imread ('/content/20201125084906.jpg') 
cv2_imshow(wkwk)